In [ ]:
import pybind11
import sys

from pybind11 import get_include

In [ ]:
inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

print(f"{inc = }\n")
print(f"{plat = }\n")
print(f"{pyinc.s = }\n")

What about classes?

In [ ]:
%%writefile VectorClass.hpp
#pragma once

class Vector2D {
    double x;
    double y;
    
public:

    Vector2D(double x, double y): x(x), y(y) {}
    
    float get_x() const {
        return x;
    }
    
    float get_y() const {
        return y;
    }
    
    void set_x(float val) {
        x = val;
    }
    
    void set_y(float val) {
        y = val;
    }

    
    Vector2D& operator+= (const Vector2D& other) {
        x += other.x;
        y += other.y;
        return *this;
    }
    
    Vector2D operator+ (const Vector2D& other) const {
        return Vector2D(x + other.x, y + other.y);
    }
};

Our binding code shows a couple more features:

In [ ]:
%%writefile vectorclass.cpp

#include <pybind11/pybind11.h>
#include <pybind11/operators.h>
#include "VectorClass.hpp"

namespace py = pybind11;
using namespace pybind11::literals;

PYBIND11_MODULE(vectorclass, m) {
    py::class_<Vector2D>(m, "Vector2D")
        .def(py::init<double, double>(), "x"_a, "y"_a)
        .def_property("x", &Vector2D::get_x, &Vector2D::set_x)
        .def_property("y", &Vector2D::get_y, &Vector2D::set_y)
        .def(py::self += py::self)
        .def(py::self + py::self)
        .def("__repr__", [](py::object self){
            return py::str("{0.__class__.__name__}({0.x}, {0.y})").format(self);
        })
    ;
}

In [ ]:
!c++ -std=c++11 vectorclass.cpp -shared {inc} {pyinc.s} -o vectorclass.so {plat}

In [ ]:
import vectorclass

v = vectorclass.Vector2D(1, 2)
print(f"{v.x = }, {v.y = }")